In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Processing results
import json

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
from sklearn.linear_model import LinearRegression

# Processing
from sklearn.preprocessing import StandardScaler

# Data imputation
from sklearn.impute import SimpleImputer

# Pipeline
from sklearn.pipeline import Pipeline

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score


# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

<IPython.core.display.Javascript object>

# Functions and definitions

In [3]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}

CEM_TYPE = ["cem_type_CEM B", "cem_type_CEM C"]

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [4]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_iv",
    "Features": "Chemical + Mineralogical + CS2 + One-Hot",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "Linear Regression",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [5]:
df = pd.read_csv("../../../../../data/processed/partner_iv/cement-shipping.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we keep only chemical and mineralogical features yielded by the same testing method/procedure

In [6]:
df_copy = df.copy()
df_copy[CEM_TYPE] = df_copy[CEM_TYPE].astype(int)
df_copy = df.drop(
    [
        # Properties
        "Blaine",
        "ph2oimm", # Maybe pH of the immersion liquid | pH (acidity or alkalinity) 
        "Initial Setting Time",
        "flow", # Maybe flow table test
        "residuo 24 micron", # Maybe the residue left after passing the material through a sieve
        "R_wp", # Maybe water to powder ratio
        "Soundness",
                
        # Chemical Composition
        # Reason: Loss on Ignition is the only feature
        # that belongs to chemical composition in which was 
        # measured by a different method, namely manual        
    ],
    axis=1,
).copy()


<IPython.core.display.Javascript object>

# 1. Linear Regression

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered. Only Chemical and mineralogical features measured by the same method. For this particular dataset, all chemical features, with the exception of LOI were measured by XRF and XRD methods.

In [7]:
y = df_copy.pop("CS28").values
x = df_copy.drop(["Date"], axis=1)
dates = df["Date"].copy()

<IPython.core.display.Javascript object>

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [8]:
repeats = 3
n_splits = 5
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Repeated KFold"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Repeated Cross Validation:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -2.097 (0.020)
MAE: -1.639 (0.019)
MAPE: -0.034 (0.000)
R2: 0.899 (0.002)


******
[TEST]
******
RMSE: -2.192 (0.092)
MAE: -1.714 (0.066)
MAPE: -0.035 (0.001)
R2: 0.889 (0.008)




<IPython.core.display.Javascript object>

In [9]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                 
0  Chemical + Mineralogical + CS2 + One-Hot  Linear Regression   

  Cross Validation RMSE Test            MAE Test           MAPE Test  \
                        mean       std      mean       std      mean   
0   Repeated KFold  2.192123  0.091618  1.714332  0.065744  0.035107   

              R2 Test            
        std      mean       std  
0  0.001422  0.889427  0.008323

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [10]:
n_splits = 5
train_size = 0.8

pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
results_dict_copy[
    "Cross Validation Params"
] = '{"N_Splits": 5, "Repeats": 1, "train_size": 0.8}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.662 (0.246)
MAE: -1.316 (0.162)
MAPE: -0.027 (0.003)
R2: 0.933 (0.020)


******
[TEST]
******
RMSE: -2.535 (0.710)
MAE: -2.089 (0.530)
MAPE: -0.042 (0.010)
R2: 0.825 (0.087)




<IPython.core.display.Javascript object>

In [11]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                 
0  Chemical + Mineralogical + CS2 + One-Hot  Linear Regression   
1  Chemical + Mineralogical + CS2 + One-Hot  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  2.535367  0.710439  2.088931  0.530090   
1              Repeated KFold  2.192123  0.091618  1.714332  0.065744   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.042289  0.009752  0.824612  0.086928  
1  0.035107  0.001422  0.889427  0.008323

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [12]:
n_splits = 5
gap = 0
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = TimeSeriesSplit(gap=gap, max_train_size=None, n_splits=n_splits, test_size=None)

scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Time Series Split"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 1, "Gap": 0}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -2.038 (0.021)
MAE: -1.576 (0.031)
MAPE: -0.032 (0.000)
R2: 0.903 (0.002)


******
[TEST]
******
RMSE: -2.632 (0.406)
MAE: -2.028 (0.257)
MAPE: -0.041 (0.006)
R2: 0.828 (0.069)




<IPython.core.display.Javascript object>

In [13]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                 
0  Chemical + Mineralogical + CS2 + One-Hot  Linear Regression   
1  Chemical + Mineralogical + CS2 + One-Hot  Linear Regression   
2  Chemical + Mineralogical + CS2 + One-Hot  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  2.535367  0.710439  2.088931  0.530090   
1              Repeated KFold  2.192123  0.091618  1.714332  0.065744   
2           Time Series Split  2.632185  0.406335  2.028096  0.256656   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.042289  0.009752  0.824612  0.086928  
1  0.035107  0.001422  0.889427  0.008323  
2  0.041425  0.006362  0.827885  0.068636

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [14]:
test_size = 0.2

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_size, random_state=SEED, shuffle=False
)
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)

pipeline.fit(x_train, y_train)

y_train_pred = pipeline.predict(x_train)
y_test_pred = pipeline.predict(x_test)

scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Out of time Split"
results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(
    results_dict_copy, {key: [value] for key, value in scores.items()}
)
results_to_save.append(df_results)

******
[TRAIN]
******
RMSE: 2.052 (0.000)
MAE: 1.606 (0.000)
MAPE: 0.033 (0.000)
R2: 0.906 (0.000)


******
[TEST]
******
RMSE: 2.557 (0.000)
MAE: 2.014 (0.000)
MAPE: 0.041 (0.000)
R2: 0.833 (0.000)




<IPython.core.display.Javascript object>

In [15]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                 
0  Chemical + Mineralogical + CS2 + One-Hot  Linear Regression   
1  Chemical + Mineralogical + CS2 + One-Hot  Linear Regression   
2  Chemical + Mineralogical + CS2 + One-Hot  Linear Regression   
3  Chemical + Mineralogical + CS2 + One-Hot  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  2.535367  0.710439  2.088931  0.530090   
1           Out of time Split  2.556731  0.000000  2.013698  0.000000   
2              Repeated KFold  2.192123  0.091618  1.714332  0.065744   
3           Time Series Split  2.632185  0.406335  2.028096  0.256656   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.042289  0.009752  0.824612  0.086928  
1  0.041141  0.000000  0.832903  0.000000  
2  0.035107  0.001422  0.889427  0.008323  
3  0.041425  0.006362  0.827885  0.068636

<IPython.core.display.Javascript object>

In [16]:
pipeline.named_steps["estimator"].coef_

array([ 1.45098091e-01, -7.74266198e-01,  2.60824032e+00, -7.78256692e+00,
       -4.30702821e+00,  3.98421491e+01, -8.05119118e-01,  3.17350151e-03,
        9.70602273e+00, -1.03866404e+00, -1.94237901e+00,  5.52951030e+00,
        6.48337674e+00,  6.55861657e+00,  5.86739289e+00, -4.34377596e+00,
        6.85577380e-01,  2.09434368e+00,  1.97454779e-01,  3.83979525e-01,
       -1.76116949e-01,  1.33970825e+00,  5.58536681e-01,  1.18627729e+00,
        3.62864083e+01,  2.21860135e+00, -5.39270086e-01, -7.22112008e-01,
       -2.80052409e+00])

<IPython.core.display.Javascript object>

In [17]:
pipeline.named_steps["estimator"].intercept_

49.225848563968675

<IPython.core.display.Javascript object>

In [18]:
coeff = pipeline.named_steps["estimator"].coef_
np.array(x.columns)[coeff == 0]

array([], dtype=object)

<IPython.core.display.Javascript object>

In [19]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns

Index(['SO3', 'Cl-', 'CS2', 'Alite_M3 C3S M3', 'Alite_M1 C3S M1',
       'Alite_Sum C3S tot', 'Ratio_M1 (rapporto M1/M3)',
       'C3S_CS (taglia dei cristalliti C3S)', 'Belite_beta', 'C3A cub',
       'C3A_ortho', 'C3A tot', 'C4AF', 'CaO', 'Ca(OH)2', 'Calce libera',
       'Periclasio (MgO)', 'Quartz', 'K2SO4', 'Langbeinite – MgK2(SO4)2',
       'Aphthitalite – (K,Na)3(SO4)2', 'Gesso', 'Emiidrato', 'Anidrite',
       'Calcite – CaCO3', 'SO3_XRD', 'CO2_XRD', 'cem_type_CEM B',
       'cem_type_CEM C'],
      dtype='object')

<IPython.core.display.Javascript object>

In [20]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns.shape

(29,)

<IPython.core.display.Javascript object>

In [21]:
x.columns.shape

(29,)

<IPython.core.display.Javascript object>

In [22]:
coeffs = pd.DataFrame(
    {col: [c] for col, c in zip(x.columns, coeff)}, index=["Coefficients"]
)

<IPython.core.display.Javascript object>

In [23]:
coeffs.T["Coefficients"].sort_values(ascending=False).to_frame(
    name="Coefficients"
).style.background_gradient(axis=None, vmin=1, vmax=5, cmap="Greens")

,Coefficients
Alite_Sum C3S tot,39.842149
Calcite – CaCO3,36.286408
Belite_beta,9.706023
CaO,6.558617
C4AF,6.483377
Ca(OH)2,5.867393
C3A tot,5.529510
CS2,2.608240
SO3_XRD,2.218601
Quartz,2.094344


<IPython.core.display.Javascript object>

In [24]:
pd.concat(results_to_save).groupby(
    ["Features", "Model", "Cross Validation", "Cross Validation Params"]
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                 
0  Chemical + Mineralogical + CS2 + One-Hot  Linear Regression   
1  Chemical + Mineralogical + CS2 + One-Hot  Linear Regression   
2  Chemical + Mineralogical + CS2 + One-Hot  Linear Regression   
3  Chemical + Mineralogical + CS2 + One-Hot  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Test            \
                                                         mean       std   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}  2.535367  0.710439   
1                                {"Test Size": 0.2}  2.556731  0.000000   
2                     {"N_Splits": 5, "Repeats": 3}  2.192123  0.091618   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}  2.632185  0.406335   

   MAE Test           MAPE Test             R2 Test            
       mean       std      mean       std      mean       std  
0  2.088931  0.530090  0.042289  0.009752  0.824612  0.086928  
1  2.013698  0.000000  0.041141  0.000000  0.832903  0.000000  
2  1.714332  0.065744  0.035107  0.001422  0.889427  0.008323  
3  2.028096  0.256656  0.041425  0.006362  0.827885  0.068636

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [25]:
path = "../../../../../reports/results/local_models/partner_iv/all_cements/full/"
filename = "linear_regression_results_full_4.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

## Saving the grouped dataframe

In [26]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../reports/results/local_models/partner_iv/all_cements/grouped/"
filename = "linear_regression_results_grouped_4.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>